In [31]:
!pip install imutils

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pandas as pd
import cv2 as cv
from imutils import paths
import skimage
from skimage.filters import sobel
from skimage import segmentation
from skimage.color import label2rgb
from skimage.color import rgb2hed, hed2rgb
from skimage.exposure import rescale_intensity
from skimage.measure import regionprops, regionprops_table
from scipy import ndimage as ndi
import imageio.v2 as imageio
from sklearn.preprocessing import StandardScaler

!pip install imutils

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import pandas as pd
import cv2 as cv
from imutils import paths
import skimage
from skimage.filters import sobel
from skimage import segmentation
from skimage.color import label2rgb
from skimage.color import rgb2hed, hed2rgb
from skimage.exposure import rescale_intensity
from skimage.measure import regionprops, regionprops_table
from scipy import ndimage as ndi
import imageio.v2 as imageio
from sklearn.preprocessing import StandardScaler

In [32]:
CE_folder_path = r'../input/mayo-compressed/MAYO/CE'
LAA_folder_path = r'../input/mayo-compressed/MAYO/LAA'

In [33]:
CE_images = sorted(list(paths.list_images(CE_folder_path)))
print('There are ' + str(len(CE_images)) + " images in the CE folder")
CE_images[:5]

In [34]:
LAA_images = sorted(list(paths.list_images(LAA_folder_path)))
print('There are ' + str(len(LAA_images)) + " images in the LAA folder")
LAA_images[:5]

**Opening an Image**

In [35]:
def read_png(path):
    image = imageio.imread(path)
    filename = path.split('/')[-1].rstrip('.png')
    print("image_id: " + filename)
    return image, filename

In [36]:
image, filename = read_png(CE_images[16])
image.shape

**Converting the Image to Grayscale**

In [37]:
def convert_image_grayscale(image):
    gray_image = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    return gray_image

In [38]:
gray_img = convert_image_grayscale(image)

In [39]:
plt.figure(figsize=(10,10))
plt.imshow(gray_img, cmap='gray')
plt.show()

**Image Segmentation**

In [40]:
def segment_images(gray_img):
    elevation_map = sobel(gray_img) # creates image (array) that emphasizes edges
    markers = np.zeros_like(gray_img) # array of zeros in the same shape as the resized_gray_img array
    markers[gray_img >= gray_img.mean()] = 1
    markers[gray_img < gray_img.mean()] = 2
    segmented_img = segmentation.watershed(elevation_map, markers) # distinguishes objects in image 
    filled_segments = ndi.binary_fill_holes(segmented_img - 1) # method works with 0s and 1s (hence the -1, now 1 = area to keep). If 0 is eclosed by 1s -> 0 converted to 1
    labeled_segments, _ = ndi.label(filled_segments) # labels features in array, where non-zero values in input = features, and zero values = background.
    return labeled_segments

In [41]:
labeled_segments = segment_images(gray_img)

In [42]:
def plot_labeled_segments(labeled_segments, gray_img):
    image_label_overlay = label2rgb(labeled_segments, image=gray_img, bg_label=0) # Returns an RGB image where color-coded labels are painted over the image.
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_title('segmentation')
    ax.imshow(image_label_overlay, cmap=plt.cm.gray) 
    ax.axis('off')

In [43]:
plot_labeled_segments(labeled_segments, gray_img)

**Object Coordinates**

In [44]:
def get_object_coordinates(labeled_segments):
    properties =['area','bbox','convex_area','bbox_area', 'major_axis_length', 'minor_axis_length', 'eccentricity']
    df = pd.DataFrame(regionprops_table(labeled_segments, properties=properties))
    standard_scaler = StandardScaler()
    scaled_area = standard_scaler.fit_transform(df.area.values.reshape(-1,1))
    df['scaled_area'] = scaled_area
    df.sort_values(by="scaled_area", ascending=False, inplace=True)
    objects = df[df['scaled_area']>=.10] # tunable -> the lower it is, the more objects in the image will get boxed (original value = 0.75)
    display(objects.head())
    object_coordinates = [(row['bbox-0'],row['bbox-1'],row['bbox-2'],row['bbox-3'] )for index, row in objects.iterrows()]
    return object_coordinates

In [45]:
object_coordinates = get_object_coordinates(labeled_segments)

In [46]:
def plot_object_coordinates(object_coordinates, image):
    fig, ax = plt.subplots(1,1, figsize=(18, 16), dpi = 80)
    for blob in object_coordinates:
        width = blob[3] - blob[1]
        height = blob[2] - blob[0]
        patch = Rectangle((blob[1],blob[0]), width, height, edgecolor='r', facecolor='none')
        ax.add_patch(patch)
        ax.imshow(image);
        ax.set_axis_off()

In [47]:
plot_object_coordinates(object_coordinates, image)

**Display Patches**

In [48]:
def plot_objects(object_coordinates, image):
    plt.figure(figsize=(10,18))
    for i in range(len(object_coordinates)):
        ax = plt.subplot(int(np.ceil(len(object_coordinates)/3)),3,i+1)
        coordinates = object_coordinates[i]
        # print(coordinates)
        object_image = image[int(coordinates[0]):int(coordinates[2]), int(coordinates[1]):int(coordinates[3])]
        plt.imshow(object_image)
        ax.axis('off')
    plt.show()

In [49]:
plot_objects(object_coordinates, image)

**Normalise Images and Create Patch Dictionary**

In [50]:
def normalize_coordinates(object_coordinates, image):
    top, bottom, left, right = object_coordinates
    left = (int(left) / image.shape[0])
    bottom = (int(bottom) / image.shape[1])
    right = int(left) + (int(right) / image.shape[0])
    top = int(bottom) + (int(top) / image.shape[1])
    
    # object_location = top, bottom, left, right
    # top, bottom, left, right = rescale_coordinates(object_location, image)
    
    return top, bottom, left, right

In [51]:
def patches_dictionary(object_coordinates, image, filename):
    patches = {}
    for i in range(len(object_coordinates)):
        coordinates = object_coordinates[i]
        normal_cords = normalize_coordinates(coordinates, image)
        patches[str(filename)+"_"+str(i+1)] = [normal_cords]
    patches = {filename:patches}
    return patches

In [52]:
patches = patches_dictionary(object_coordinates, image, filename)
patches